In [ ]:
import os
from loguru import logger
import glob
from IPython.display import Image, display

from ultralytics import YOLO
import torch
from pennylane import numpy as np

os.chdir("..")
logger.info(f"Current directory: {os.getcwd()}")
HOME = os.getcwd()

from src.YOLO.yolo_funcs import quantum_predict
from src.nn.models.hybrid.HQNN_quanv import FlexHybridCNN
from src.utils.run_conf import CargaConfH

# Importing the Hojas Dataset

In [ ]:
HOME = HOME + "/dataset"
%cd {HOME}

model = YOLO(f"{HOME}/best.pt")

In [ ]:
results = model.predict(
    source=HOME + "/HojasSimulador",
    save=False,
    project="classic_results",
    name="inferencia_test",
    exist_ok=True
)

# Validate Custom Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

In [ ]:
image_size = 32
n_classes = 2
conf = CargaConfH(image_size,)
name = "2hojas_HybridCNN.pth"

In [ ]:
modelq = FlexHybridCNN(embedding_params = conf["embedding"],
                                    variational_params= conf["ansatz"],
                                    measurement_params= conf["measurement"],
                                    n_classes = n_classes,
                                    use_quantum = True,
                                    qkernel_shape = 3,
                                    epochs = 30,
                                    dataset = "2hojas",
                                    image_size = image_size,
                                )

modelq.to(device)
modelq.load_state_dict(torch.load(name))
modelq.eval()

In [ ]:
quantum_predict(modelq, results, image_size, "Hojas")

In [ ]:
for image_path in glob.glob(f"{os.getcwd()}/quantum_results/_hojas_lejosH/*.png")[:]:
    display(Image(filename=image_path, width=600))
    print(image_path)

for image_path in glob.glob(f"{os.getcwd()}/quantum_results/_hojas_lejosH/*.jpeg")[:]:
    display(Image(filename=image_path, width=600))
    print(image_path)

for image_path in glob.glob(f"{os.getcwd()}/quantum_results/_hojas_lejosH/*.jpg")[:]:
    display(Image(filename=image_path, width=600))
    print(image_path)